In [ ]:
import   os
import   sys
sys.path.insert(0, os.path.abspath('.'))
from     ismrmrdUtils   import   rawMRutils
import   numpy          as       np
import   ismrmrd

## First - read in (fully sampled) EPI data from array coil

In [ ]:
dataHeader, dataArray, refDataArray = rawMRutils.returnHeaderAndData('./ScanArchive_EPI_converted.h5')

In [ ]:
imageSpace = np.fft.fftshift(np.fft.fft2(dataArray, axes=(1,2)), axes=(1,2))

In [ ]:
rawMRutils.computeAndPlot(imageSpace)

In [ ]:
dataArray.shape

In [ ]:
rawDataArray = ismrmrd.Dataset('./ScanArchive_EPI_converted.h5', '/dataset', True)

In [ ]:
rawDataArray.number_of_acquisitions()

In [ ]:
for i in range(rawDataArray.number_of_acquisitions()):
   thisAcq = rawDataArray.read_acquisition(i)
   if (thisAcq.is_flag_set(24)): # 24 == ACQ_IS_PHASECORR_DATA flag - need to figure out how to include definition
      print ("Flags for acquisition %05d is %s, with phase encode step %d" % (i, thisAcq.flags, thisAcq.idx.kspace_encode_step_1))

In [ ]:
dataArrayHeader = ismrmrd.xsd.CreateFromDocument(rawDataArray.read_xml_header())

In [ ]:
print(str(dataArrayHeader.toxml()))

In [ ]:
dataArrayHeader.encoding[0].trajectoryDescription.userParameterLong[:]

In [ ]:
dataArrayHeader.encoding[0].trajectory

In [ ]:
dataArrayHeader.encoding[0].trajectoryDescription.identifier

In [ ]:
for i, val in enumerate(dataArrayHeader.encoding[0].trajectoryDescription.userParameterLong[:]):
   print ("User element %d is %18s, value: %s" % (i, dataArrayHeader.encoding[0].trajectoryDescription.userParameterLong[i].name, val))

In [ ]:
dataArrayHeader.encoding[0].trajectoryDescription.userParameterLong[1].value